In [1]:
import pandas as pd
import os
import numpy as np

#Takeaway 1: The amount of school funder per student does not correlate to the pass rates for the students at that school.
#Takeaway 2: Charter schools outperform district schools in this sample both for math pass rates and reading pass rates
#Takeaway 3: Small schools outperform medium and large schoolsfor this sample in both math and reading pass rates

In [2]:
schools_csv_path = os.path.join('..', 'raw_data', 'schools_complete.csv')
schools_df = pd.read_csv(schools_csv_path)

In [3]:
students_csv_path = os.path.join('..', 'raw_data', 'students_complete.csv')
students_df = pd.read_csv(students_csv_path)

In [4]:
merged_data_df = pd.merge(schools_df,students_df,left_on='name', right_on='school',how='outer')

In [5]:
merged_data_df.rename(index=str,columns={'name_x':"School Master",'name_y':'Student'},inplace=True)

In [6]:
grouped_data = merged_data_df.groupby('School Master')

In [7]:
total_students = merged_data_df['Student'].count()
total_budget = grouped_data['budget'].unique()
total_budget = total_budget.astype("int")
total_grouped_students = grouped_data['Student'].count()
per_student_budget = total_budget/total_grouped_students
average_math = grouped_data['math_score'].sum()/total_grouped_students
average_reading = grouped_data['reading_score'].sum()/total_grouped_students

In [8]:
math_pass_percent = merged_data_df[merged_data_df['math_score']>69].groupby('School Master').count()
math_pass_percent = math_pass_percent['math_score']
math_pass_percent = math_pass_percent/total_grouped_students

In [9]:
reading_pass_percent = merged_data_df[merged_data_df['reading_score']>69].groupby('School Master').count()
reading_pass_percent = reading_pass_percent['reading_score']
reading_pass_percent = reading_pass_percent/total_grouped_students

In [10]:
total_pass_percent = (math_pass_percent+reading_pass_percent)/2

In [11]:
school_type = grouped_data['type'].unique()

In [25]:
school_summary_df = pd.DataFrame({'School Type':school_type,
                                  'Total Students':total_grouped_students,
                                  'Total School Budget':total_budget,
                                 'Per Student Budget':per_student_budget,
                                 'Average Math Score':average_math,
                                 'Average Reading Score':average_reading,
                                 'Percent Pass Math':math_pass_percent,
                                 'Percent Pass Reading':reading_pass_percent,
                                 'Total Pass Percent':total_pass_percent})

school_summary_df.head(1)

,Average Math Score,Average Reading Score,Per Student Budget,Percent Pass Math,Percent Pass Reading,School Type,Total Pass Percent,Total School Budget,Total Students
School Master,,,,,,,,,
Bailey High School,77.048432,81.033963,628.0,0.666801,0.819333,[District],0.743067,3124928,4976


In [13]:
district_total_schools = merged_data_df.groupby('School Master').nunique()
district_total_schools = district_total_schools['Student ID'].count()

district_total_students = total_students
district_total_budget = total_budget.sum()

district_avg_math = merged_data_df['math_score'].mean()
district_avg_reading = merged_data_df['reading_score'].mean()

district_math_pass_pct = (merged_data_df[merged_data_df['math_score']>69].count())/district_total_students
district_reading_pass_pct = (merged_data_df[merged_data_df['reading_score']>69].count())/district_total_students

district_math_pass_pct = district_math_pass_pct['Student ID']
district_reading_pass_pct = district_reading_pass_pct['Student ID']

dist_pass_rate = (district_math_pass_pct+district_reading_pass_pct)/2
dist_pass_rate*100

80.393158029103901

In [14]:
district_summary_df = pd.DataFrame({'Total Schools':district_total_schools,
                                    'Total Students':district_total_students,
                                    'Average Math Score':district_avg_math,
                                   'Average Reading Score':district_reading_pass_pct,
                                   'Math Pass Percentage':district_math_pass_pct,
                                   'Reading Pass Percentage': district_reading_pass_pct,
                                   'Total Pass Rate':dist_pass_rate},index=['District'])

In [15]:
district_summary_df

,Average Math Score,Average Reading Score,Math Pass Percentage,Reading Pass Percentage,Total Pass Rate,Total Schools,Total Students
District,78.985371,0.858055,0.749809,0.858055,0.803932,15,39170


In [16]:
top_schools_df = school_summary_df.sort_values(by=['Total Pass Percent'],ascending=False)
top_schools_df.head(1)

,Average Math Score,Average Reading Score,Per Student Budget,Percent Pass Math,Percent Pass Reading,School Type,Total Pass Percent,Total School Budget,Total Students
School Master,,,,,,,,,
Cabrera High School,83.061895,83.97578,582.0,0.941335,0.970398,[Charter],0.955867,1081356,1858


In [17]:
bottom_schools_df = school_summary_df.sort_values(by=['Total Pass Percent'])
bottom_schools_df.head(1)

,Average Math Score,Average Reading Score,Per Student Budget,Percent Pass Math,Percent Pass Reading,School Type,Total Pass Percent,Total School Budget,Total Students
School Master,,,,,,,,,
Rodriguez High School,76.842711,80.744686,637.0,0.663666,0.802201,[District],0.732933,2547363,3999


In [18]:
grouped_data_grade_df = merged_data_df.groupby(['School Master','grade'])
grouped_data_grade_math = grouped_data_grade_df['math_score'].mean()
grouped_data_grade_math.head(3)

School Master       grade
Bailey High School  10th     76.996772
                    11th     77.515588
                    12th     76.492218
Name: math_score, dtype: float64

In [19]:
grouped_data_grade_reading = grouped_data_grade_df['reading_score'].mean()
grouped_data_grade_reading.head(3)

School Master       grade
Bailey High School  10th     80.907183
                    11th     80.945643
                    12th     80.912451
Name: reading_score, dtype: float64

In [20]:
bins = [0, 600, 625, 650, 675]
group_names = ['critical', 'low', 'moderate', 'high']
school_summary_df['funding category'] = pd.cut(school_summary_df["Per Student Budget"],bins, labels=group_names)

school_funding_summary_df = school_summary_df.groupby(['funding category'])
school_funding_summary_df = school_funding_summary_df.mean()

In [21]:
school_funding_summary_df.drop(columns=['Total Students','Total School Budget'])

,Average Math Score,Average Reading Score,Per Student Budget,Percent Pass Math,Percent Pass Reading,Total Pass Percent
funding category,,,,,,
critical,83.436210,83.892196,584.800000,0.935415,0.964596,0.950006
low,83.595708,83.930728,617.000000,0.939935,0.965425,0.952680
moderate,78.032719,81.416375,639.333333,0.711124,0.834538,0.772831
high,76.959583,81.058567,653.500000,0.662184,0.810897,0.736541


In [22]:
bins = [0, 1250, 2501, 5001]
group_names = ['Small', 'Medium', 'Large']
school_summary_df['school size'] = pd.cut(school_summary_df["Total Students"],bins, labels=group_names)
school_size_df = school_summary_df.groupby(['school size'])
school_size_df = school_size_df.mean()

In [23]:
school_size_df.drop(columns=['Total Students','Total School Budget'])

,Average Math Score,Average Reading Score,Per Student Budget,Percent Pass Math,Percent Pass Reading,Total Pass Percent
school size,,,,,,
Small,83.821598,83.929843,595.000000,0.935502,0.960994,0.948248
Medium,83.357937,83.885280,601.000000,0.936444,0.967488,0.951966
Large,76.956733,80.966636,643.571429,0.665485,0.807991,0.736738


In [24]:
school_type_df = school_summary_df
school_type_df = school_type_df.drop(columns=['Total Students', 'Total School Budget'])
school_type_df['School Type'] = school_type_df['School Type'].astype(str)
school_type_df = school_type_df.groupby(['School Type'])
school_type_df.mean()

,Average Math Score,Average Reading Score,Per Student Budget,Percent Pass Math,Percent Pass Reading,Total Pass Percent
School Type,,,,,,
['Charter'],83.473852,83.896421,599.500000,0.936208,0.965865,0.951037
['District'],76.956733,80.966636,643.571429,0.665485,0.807991,0.736738
